<a href="https://colab.research.google.com/github/dominiksakic/zero_to_hero/blob/main/basics_06_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- source: https://www.youtube.com/watch?v=kCc8FmEb1nY&t=126s

In [ ]:
# get data

In [ ]:
# make decoder, encoder

In [ ]:
# create a baseline model/bigram

In [ ]:
# train the baseline

# Transformers


In [ ]:
# Gensis of Transformer
# Averaging past context with for loops
# Matrix multiplying as weighted aggregation
# Final version